In [7]:
import pandas as pd
import numpy as np
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import transforms

In [14]:
PATH_TABULAR_DATA = "real_estate_price_prediction_processed_09042024.csv"

In [12]:
!pwd

/Users/cesar/github/bourbaki_challenges/evaluacion


In [13]:
!ls

00_eda_reto_bourbaki.ipynb
01_model_tabular_reto_bourbaki.ipynb
02_model_image_embbeding_reto_bourbaki-Copy1.ipynb
02a_model_image_embbeding_reto_bourbaki.ipynb
real_estate_price_prediction_metadata_images_09042024.csv
real_estate_price_prediction_processed_09042024.csv
reduced_images


In [15]:
data_tab = pd.read_csv(PATH_TABULAR_DATA)

In [16]:
data_tab[["price_scaled"]] = MinMaxScaler().fit_transform(data_tab[["price"]])

In [17]:
data_tab.head()

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,price,price_scaled
0,35996577,0,43.643880,7.117183,8452,199,63.0,3.479524,3995.665362,205.385148,...,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,355000.0,0.145320
1,35811033,0,45.695757,4.895610,8234,3347,90.0,3.000000,3995.665362,223.000000,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190000.0,0.072778
2,35731841,12,47.966791,-1.220451,5235,1650,61.0,3.479524,370.000000,205.385148,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39000.0,0.006390
3,35886765,12,47.289292,-1.878805,1929,2050,142.0,3.479524,764.000000,217.000000,...,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,299000.0,0.120699
4,35781137,0,45.718992,4.844234,4476,3334,88.0,3.000000,3995.665362,205.385148,...,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,478000.0,0.199397


In [18]:
X_train_, X_test, y_train_, y_test= train_test_split(
    data_tab.drop(columns=["price", "price_scaled"]),
    data_tab["price_scaled"],
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [19]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_,
    y_train_,
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [21]:
X_reorganized = pd.concat(
    [
        pd.DataFrame(X_train).assign(group="train"),
        pd.DataFrame(X_val).assign(group="val"),
        pd.DataFrame(X_test).assign(group="test")
    ]
).reset_index(drop=True)


y_reorganized = pd.concat(
    [
        pd.DataFrame(y_train).assign(group="train"),
        pd.DataFrame(y_val).assign(group="val"),
        pd.DataFrame(y_test).assign(group="test")
    ]
).reset_index(drop=True)

In [22]:
X_reorganized.sample(5)

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,group
6757,35851649,0,43.532040,1.237867,2638,1409,67.0,3.479524,3995.665362,205.385148,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
21590,35795069,12,48.971530,2.284296,6729,4668,153.0,3.479524,840.000000,326.000000,...,1.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train
25527,35751703,12,43.244750,2.864573,8240,366,72.0,3.479524,3995.665362,205.385148,...,0.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,val
9504,35939869,5,48.680117,2.379610,3298,4480,130.0,3.479524,3995.665362,205.385148,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
483,35793809,0,48.971906,2.322882,2375,4725,39.0,2.000000,3995.665362,205.385148,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train


In [11]:
#X_reorganized[["id_annonce", "group"]]

In [23]:
PATH_IMAGES_DATA = "real_estate_price_prediction_metadata_images_09042024.csv"

In [24]:
data_img = pd.read_csv(PATH_IMAGES_DATA)

In [25]:
data_img_agg = data_img.groupby('id_annonce')['long_path_image'].apply(list).reset_index(name='images')

In [26]:
data_img_groups = data_img_agg.merge(
    X_reorganized[["id_annonce", "group"]],
    how='left',
    on='id_annonce'
)

In [27]:
data_img_groups.sample(10)

,id_annonce,images,group
1237,34991273,[./reduced_images/train/ann_34991273/09de71a50...,train
26404,35997827,[./reduced_images/train/ann_35997827/f3fc52ac5...,train
2797,35752815,[./reduced_images/train/ann_35752815/69331edc6...,train
17615,35864213,[./reduced_images/train/ann_35864213/e64af3fe2...,val
5978,35780261,[./reduced_images/train/ann_35780261/ea8846f69...,train
1175,34853209,[./reduced_images/train/ann_34853209/e9076ffe2...,train
8227,35798705,[./reduced_images/train/ann_35798705/4fe3f31d9...,val
7045,35787321,[./reduced_images/train/ann_35787321/1cdf0e953...,train
9064,35806941,[./reduced_images/train/ann_35806941/579449251...,train
3867,35761811,[./reduced_images/train/ann_35761811/5707b7067...,val


In [28]:
# Define the image transformation
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ),
])


In [61]:
class CustomDataset(Dataset):

    def __init__(
        self,
        data_features,
        data_target,
        data_img_groups,
        group='train',
        transform=None
        ) -> None:

        self.group = group

        self.data_features = data_features.query(
            f"group == '{self.group}'"
            )

        self.data_target = data_target.query(
            f"group == '{self.group}'"
            )

        self.data_img_groups = data_img_groups.query(
            f"group == '{self.group}'"
            )

        self.transform = transform

        self.model = models.resnet152(pretrained=True)
        self.model_layers = list(self.model.children())[:-1]
        self.model_feature_extract = torch.nn.Sequential(*self.model_layers)

        self.set_model_eval()

    def __len__(self):
        return len(self.data_features)

    def set_model_eval(self):
      """
      Set to evaluation model.
      """
      return self.model.eval()


    def get_model_embedding(self, img_tensor):

      with torch.no_grad():
          # Get the image features from the ResNet-152 model
          img_features = self.model_feature_extract(img_tensor)

      return img_features

    def load_image(self, path) -> Image.Image:

        return Image.open(path).convert("RGB")

    def __getitem__(self, index):

        images_path = self.data_img_groups.iloc[index]["images"]

        images_emmbeded = []

        for path in images_path:
          image = self.load_image(path)

          if self.transform:
            image = self.transform(image).unsqueeze(0)

            img_embedding = self.get_model_embedding(image)
            images_emmbeded.append(img_embedding.flatten())

        #images_embbeding = torch.mean(
        #                  torch.Tensor(np.array(images_emmbeded)),
        #                  axis = 0
        #                  )

        images_embbeding = torch.stack(images_emmbeded).mean(dim=0)

        features = self.data_features.drop(
            columns=["id_annonce","group"]
            ).iloc[index,:]

        #features = torch.from_numpy(features).float()

        target = self.data_target[["price_scaled"]].values[index]
        #target = np.array(target)
        #target = torch.from_numpy(target).float()

        #sample = {}

        avg_embbeding = images_embbeding
        tab_features = torch.tensor(features.values, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)

        return avg_embbeding, tab_features, target

In [62]:
train_dataset = CustomDataset(
        data_features=X_reorganized.head(100),
        data_target=y_reorganized.head(100),
        data_img_groups=data_img_groups,
        group='train',
        transform=preprocess
    )

val_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='val',
        transform=preprocess
)

test_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='test',
        transform=preprocess
)

/Users/cesar/.pyenv/versions/miniconda3-4.7.12/envs/reto_pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/cesar/.pyenv/versions/miniconda3-4.7.12/envs/reto_pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/cesar/.pyenv/versions/miniconda3-4.7.12/envs/reto_pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings

In [63]:
BATCH_SIZE= 20 #1024

In [64]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [68]:
len(train_loader.dataset.__getitem__(1)[0])

2048

In [ ]:
train_loader.dataset.__getitem__(1)

In [89]:
# Define the neural network
class PricePredictionModel(nn.Module):
    def __init__(self, embedding_size=2048, tabular_input_size=26):  # Adjust tabular_input_size based on your actual features
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(embedding_size+tabular_input_size, 512)  # Output a single value for price prediction
        self.fc2 = nn.Linear(512, 1)

    def forward(self, image_inputs, tabular_inputs):
        combined_embedding = torch.cat((image_inputs, tabular_inputs), dim=1)
        output = self.fc1(combined_embedding)
        output = self.fc2(output)
        return output.squeeze(1) 

In [90]:
# Hyperparameters
batch_size = 16
learning_rate = 0.001
num_epochs = 10

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [93]:
# Initialize the model, loss, and optimizer
model = PricePredictionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Training loop
for epoch in range(num_epochs):
    for batch_idx, (avg_embbeding, tab_features, target) in enumerate(train_loader):
        avg_embbeding, tab_features, target = avg_embbeding.to(device), tab_features.to(device), target.to(device)

        optimizer.zero_grad()
        outputs = model(avg_embbeding, tab_features)
        loss = criterion(outputs, target.squeeze(1))
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 1 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [1/5], Loss: 1901.5789
Epoch [1/10], Step [2/5], Loss: 576.6666
Epoch [1/10], Step [3/5], Loss: 669.2802
Epoch [1/10], Step [4/5], Loss: 523.3038
Epoch [1/10], Step [5/5], Loss: 468.4623
Epoch [2/10], Step [1/5], Loss: 211.5016
Epoch [2/10], Step [2/5], Loss: 1364.6394
Epoch [2/10], Step [3/5], Loss: 877.6251
Epoch [2/10], Step [4/5], Loss: 385.6054
Epoch [2/10], Step [5/5], Loss: 306.0905
Epoch [3/10], Step [1/5], Loss: 1112.4033
Epoch [3/10], Step [2/5], Loss: 130.9079
Epoch [3/10], Step [3/5], Loss: 369.2668
Epoch [3/10], Step [4/5], Loss: 363.7661
Epoch [3/10], Step [5/5], Loss: 639.0367
Epoch [4/10], Step [1/5], Loss: 231.0081
Epoch [4/10], Step [2/5], Loss: 454.8429
Epoch [4/10], Step [3/5], Loss: 51.4064
Epoch [4/10], Step [4/5], Loss: 99.5666
